# load

> Module for loading training and target files

In [ ]:
#| default_exp load

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
import pkg_resources
from sepia.SepiaData import SepiaData

In [ ]:
#| hide

import matplotlib.pylab as plt
import os
from scipy.interpolate import interp1d


In [ ]:
#| export

DATA_DIR = "data/pkg_data/"
LIBRARY_PARAM_FILE = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "params_latin.txt").name
LIBRARY_PK_FILE = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "pk_all.npy").name
LIBRARY_K_FILE = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "k_all.npy").name
LIBRARY_Z_FILE = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "z_all.npy").name


PARAM_NAME = [r"$\Omega_m$", r"h", r"$\sigma_8$", r"$\log(z_c)$", r"$f_{ede}$", r"$\theta_i$", r"$\Omega_{CDM}$"]

# LIBRARY_ZK_FILE_VAL = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "z_k_validation.txt").name
# LIBRARY_BK_FILE_VAL = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "Boost_validation.npy").name
# LIBRARY_PARAM_FILE_VAL = pkg_resources.resource_stream("EarlyDarkEmu", DATA_DIR + "cosmo_validation.txt").name


In [ ]:
#| hide

def load_single_pk(file_path
                   ):
    data = np.loadtxt(file_path).T
    return data

In [ ]:
#| hide

def interpolate_data(k, 
                     pk, 
                     k_base
                     ):
    interp_func = interp1d(k, pk, kind='linear', bounds_error=False, fill_value="extrapolate")
    pk_interpolated = interp_func(k_base)
    return pk_interpolated

In [ ]:
#| hide

def extract_z_value(file_path
                    ):
    with open(file_path, 'r') as file:
        first_line = file.readline()
        z_value = float(first_line.split('z=')[1].split()[0])
    return z_value

In [ ]:
#| hide

def load_all_save_npy(directory, 
                       k_base, 
                       output_directory,
                       total_sim_num=64, 
                       total_redshift_num=51, 
                       ):
    
    all_data = np.zeros((total_sim_num, total_redshift_num, len(k_base)))
    all_z = np.zeros(total_redshift_num, )

    for sim_num in range(1, total_sim_num + 1):
        sim_data = []
        
        
        for redshift_num in range(1, total_redshift_num + 1):
            file_name = f"EDE_model{sim_num}_00_z{redshift_num}_pk.dat"
            file_path = os.path.join(directory, file_name)
            k, pk = load_single_pk(file_path)
            # print(k.min(), k.max())
            pk_interpolated = interpolate_data(k, pk, k_base)            
            sim_data.append(pk_interpolated)


            if sim_num == 1: 
                
                z_single = extract_z_value(file_path)
                all_z[redshift_num -1] = z_single

        
        all_data[sim_num - 1, :, :] = np.array(sim_data)  # Adjusted index to be 0-based

        np.save(output_directory + 'pk_all', all_data, allow_pickle=True)
        np.save(output_directory + 'k_all.npy', k_base)
        np.save(output_directory + 'z_all.npy', all_z)
    
    return all_data, k_base, all_z


In [ ]:
#| hide 

k_base = np.logspace(-5,  0, 210)
print(k_base.shape)

(210,)


In [ ]:
#| hide

if_local_processing = False

if if_local_processing:
    
    file_dir = '/home/nramachandra/Projects/MG_emu/Data/raw_runs/LatinHypercubeRuns_AllRedshifts/'
    out_dir = '/home/nramachandra/Projects/MG_emu/Codes/EarlyDarkEmu/EarlyDarkEmu/data/pkg_data/'
    Pk_all, k_all, z_all = load_all_save_npy(file_dir, 
                                            output_directory = out_dir, 
                                            k_base=k_base)

In [ ]:
#| export

def load_npy_pk_k_z(Pk_fileIn:str=LIBRARY_PK_FILE, # Input file for Pk
                     k_fileIn:str=LIBRARY_K_FILE,  # Input file for k
                     z_fileIn:str=LIBRARY_Z_FILE, # Input file for z
                     ) -> tuple:#Three n-D arrays for P(k), k and z


    Pk_all = np.load(Pk_fileIn, allow_pickle=True)
    k_all = np.load(k_fileIn)
    z_all = np.load(z_fileIn)

    return Pk_all, k_all, z_all


In [ ]:
#| hide

if if_local_processing: 
    Pk_all, k_all, z_all = load_npy_pk_k_z(LIBRARY_PK_FILE, LIBRARY_K_FILE, LIBRARY_Z_FILE)

In [ ]:
#| hide

if if_local_processing:

    print(np.where(Pk_all == np.max(Pk_all)))

    remove_sim_indx = [3]
    valid_indices = [i for i in  np.arange(Pk_all.shape[0])  if i not in remove_sim_indx]

    Pk_all_clean = Pk_all[valid_indices]


    plt.plot(k_base, Pk_all[:, 49, :].T, 'k');
    plt.plot(k_base, Pk_all_clean[:, 49, :].T, 'r');
    plt.xscale('log')
    plt.yscale('log')

In [ ]:
#| hide

if if_local_processing:

    file_dir = '/home/nramachandra/Projects/MG_emu/Data/raw_runs/LatinHypercubeRuns_AllRedshifts/'

    file_pk_trial = 'EDE_model12_00_z43_pk.dat'
    pk, k = load_single_pk(file_dir + file_pk_trial)

    file_pk_trial = 'EDE_model50_00_z43_pk.dat'
    pk2, k2 = load_single_pk(file_dir + file_pk_trial)

    print(k.shape, k2.shape)

    # plt.plot(k, k2[1:], 'x')
    plt.plot(pk, k)
    plt.plot(pk2, k2)
    plt.xscale('log')
    plt.yscale('log')


In [ ]:
# #| export

# def load_boost_data(Bk_fileIn:str=LIBRARY_BK_FILE, # Input file for Boost
#                         Zk_fileIn:str=LIBRARY_ZK_FILE, # Input file for redshift and wavenumbers
#                         ) -> tuple: # Boost, wavenumbers, redshifts 
#     Bk_all = np.load(Bk_fileIn)
#     zk_all = np.loadtxt(Zk_fileIn)
    
#     z_all = zk_all[:, 0][np.isfinite(zk_all[:, 0])]
#     k_all = zk_all[:, 1]
    
#     return Bk_all, k_all, z_all

In [ ]:
#| export

def load_params(p_fileIn:str=LIBRARY_PARAM_FILE, # Input file for parameters
               ) -> np.array: # Parameters
    p_all = np.loadtxt(p_fileIn)
    # p_all[:, 2] = p_all[:, 2]/1e-9  # A_s rescaling
    return p_all[:, 1:]

In [ ]:
#| hide

if if_local_processing:
    # Bk_all, k_all, z_all = load_boost_data()
    p_all = load_params()
    print(p_all[3])

In [ ]:
#| export

def sepia_data_format(design:np.array=None, # Params array of shape (num_simulation, num_params)
                     y_vals:np.array=None, # Shape (num_simulation, num_y_values)
                     y_ind:np.array=None # Shape (num_y_values,)
                     ) -> SepiaData: #Sepia data format
    sepia_data = SepiaData(t_sim=design, y_sim=y_vals, y_ind_sim=y_ind)
    return sepia_data

In [ ]:
#| hide

if if_local_processing:

    z_index = 0
    y_vals = Pk_all[:, z_index, :]
    y_ind = np.arange(0, y_vals.shape[1])
    sepia_data = sepia_data_format(p_all, y_vals, y_ind)
    print(sepia_data)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()